# Running RRTMGP on Garand Atmospheres

## Import Libraries and Modules

In [ ]:
import os, sys, shutil
import subprocess as sub
import netCDF4 as nc
import numpy as np

sys.path.append('common')
import utils

## Running the Model on Garand Atmospheres

1. Set some runtime variables that tell us about our configuration -- what spectral domain are we plotting, what cost function and normalization technique are we using, what _g_-points are we combining (only adjacent _g_-points are combined in a given iteration, so only the first _g_-point needs to be assigned)
2. Grab corresponding _k_-distribution file (provided by Menno and stored in Rick's scratch space) and stage it for RRTMGP
3. Link to the RRTMGP driver for Garand atmospheres (stored in E3SM project space)
4. Copy the LBLRTM reference netCDF into the working directory (its gas optics parameters will be used in the model). Also, apparently Menno and I have been modeling with a non-optimized diffusivity angle. One angle is used in RRMTGP, but for this exercise we should be using the 3 from Gaussian Quadrature. So we have to replace the `angle` parameter before running RRTMGP.
5. Use items 2 and 4 as input into the driver and run the it. Fluxes and heating rate variables in 4 will be overwritten.

In [ ]:
sw = False
iCost = 1
iNorm = 1
iteration = 0
gpt1 = 0

domain = 'sw' if sw else 'lw'
costStr = '{}.cost{:02d}.norm{:1d}'.format(domain, iCost, iNorm)
scratch = '/global/cscratch1/sd/pernak18/RRTMGP/g-point-reduction/data'
project = '/project/projectdirs/e3sm/pernak18/g-point-reduction/k-distribution-opt'
coeffNC = '{}/{}/coefficients_iter{:03d}.{:03d}.{:03d}.nc'.format(
    scratch, costStr, iteration, gpt1, gpt1+1)

exe = '{}/rrtmgp_garand_atmos'.format(project)

paths = [coeffNC, exe]

# link to k-distribution for this iteration/g-point combination
kDistNC = 'coefficients_{}.nc'.format(domain)
if os.path.islink(kDistNC): os.unlink(kDistNC)
os.symlink(coeffNC, kDistNC)

# copy LBLRTM reference netCDF with Garand specs
# 'test_RRTMGP_fluxes.nc' will be overwritten if it already exists
garandNC = '/project/projectdirs/e3sm/pernak18/inputs/g-point-reduce/' + \
    'lblrtm-lw-flux-inputs-outputs-garandANDpreind.nc'
outNC = 'test_RRTMGP_fluxes.nc'
shutil.copyfile(garandNC, outNC)

# link to Garand atmosphere driver for RRTMGP
baseExe = os.path.basename(exe)
if os.path.islink(baseExe): os.unlink(baseExe)
os.symlink(exe, baseExe)

# run RRTMGP on Garand atmospheres and provide full path to output file
status = sub.call(['./{}'.format(baseExe), outNC, kDistNC])
if status == 0: print('RRTMGP fluxes have been written to {}'.format(
    os.path.abspath(outNC)))

The new flux file can be used in `LW_profile_stats_plots.ipynb`

## Heating Rates

Heating rates are not calculated in RRTMGP, so we will calculate them here and store them in the netCDF file we just modified with the RRTMGP Garand driver run.

What I have found is that:

  - the broadband HRs for LBL and RRTMGP match pretty well, but they are not identical
  - by-band HRs do not match well (the 30-40% differences we see in the profile plots)
  - broadband HRs in the profile plots look as bad as the by-band HRs, but that's because we do not use the broadband arrays in the netCDF; rather, we integrate the by-band HRs band plot these as broadband

So I should just replace the by-band heating rates, and comparing their sum with the "original" RRTMGP broadband would be a good form of verification.

In [ ]:
bandNet = 'band_flux_net'
broadNet = 'flux_net'
with nc.Dataset(garandNC, 'r') as refNC, nc.Dataset(outNC, 'r') as testNC:
    netBandLBL = np.array(refNC.variables[bandNet])
    netBandGP = np.array(testNC.variables[bandNet])
    refHR = np.array(refNC.variables['band_heating_rate'])
    testHR = np.array(testNC.variables['band_heating_rate'])

    # pressures should be the same for LBL and RRTMGP
    pLev = np.array(refNC.variables['p_lev']) / 100

# deltas along the `lev` axis
iLev = 1

# for easier arithmetic, make the dP array the same dimension as 
# the dNet arrays, repeating the dP 3-D array nBand times
dP = -np.diff(pLev, axis=iLev)
dP = dP[:, :, :, np.newaxis]
dP = np.repeat(dP, 16, axis=3)

hrBandLBL = 8.4391 * np.diff(netBandLBL, axis=iLev) / dP
hrBandGP = 8.4391 * np.diff(netBandGP, axis=iLev) / dP

# sanity checks
#diff = hrBandLBL-hrBandGP
#print(hrBandLBL[0,:,0,3])
#print(refHR[0,:,0,3]*86400)
#print(hrBandLBL[0,:,0,3])
#print(hrBandGP[0,:,1,3])
#print(diff[0,:,0,3])

# replace by-band heating rates in RRTMGP netCDF
# want it back in K/day to be consistent with what plotting scripts expect
with nc.Dataset(outNC, 'r+') as testNC:
    hr = testNC.variables['band_heating_rate']
    hr[:] = hrBandGP / 86400

print('`band_heating_rate` replaced in {}'.format(outNC))